In [6]:
pip install huggingface_hub --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 8.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.27.0
    Uninstalling huggingface-hub-0.27.0:
      Successfully uninstalled huggingface-hub-0.27.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
abstract = """This work delves into the potential of applying
Generative AI techniques to the system design phase, aiming
to streamline processes and augment human expertise. Gen AI
is used to automate the creation of complex design elements and
is emerging as a powerful tool for system engineers. Also, with
LLM-generated content, evaluating and verifying the correctness
of the responses is a challenge. The SE Assistant designed
for system engineers intends to create detailed system design
documents quickly and accurately along with its evaluation.
At the core of the SE Assistant is a sophisticated system that
combines the power of GPT-4 with a Multimodal Retrieval
Augmented Generation (RAG) pipeline capable of understanding
text, images, and tables to provide valuable context. The evaluation uses the strength of strong LLMs in analyzing content
based on design-specific criteria. The SE Assistant prototype
demonstrates its ability to streamline the system design process,
from initial data gathering to the final design output, making it
an invaluable tool for system engineers.
Index Terms—Generative AI , System Engineering, System Design, Large Language Model, Retrieval Augmented Generation"""

In [10]:
from huggingface_hub import InferenceClient

def create_flowchart_prompt(abstract):
    prompt = """You are an expert in creating detailed, realistic flowcharts using Mermaid.js syntax.
Given the following text, create a comprehensive flowchart that captures all key relationships and processes.

Follow these precise guidelines:

1. RELATIONSHIP MAPPING:
   - Identify all direct and indirect relationships between elements
   - Use appropriate connection types:
     * -->: For direct sequential flow
     * -.->: For indirect/conditional relationships
     * ==>: For emphasized/major relationships
     * --o: For optional relationships
     * --x: For alternative paths

2. PROCESS STRUCTURE:
   - Use clear node types:
     * [Rectangle]: Main processes and actions
     * (Rounded Rectangle): Sub-processes
     * {Hexagon}: Input/Output points
     * <Diamond>: Decision points with multiple outcomes
     * ((Circle)): Start/End points

3. VISUAL ORGANIZATION:
   - Group related processes using subgraph
   - Maintain clear hierarchical structure
   - Use meaningful node IDs (e.g., 'process_1' instead of 'A')
   - Add descriptive labels to all connections

4. ADVANCED FEATURES:
   - Use click events for interactive nodes when relevant
   - Implement proper styling (e.g., colors, line styles)
   - Add annotations for complex relationships
   - Include tooltips for additional context

Example Input: "Describe the software deployment process."

Example Output:
```mermaid
graph TD
    subgraph Development
        start((Start)) --> code[Write Code]
        code --> test{Unit Tests}
        test -->|Failed| code
        test -->|Passed| review[Code Review]
    end
    
    subgraph Deployment
        review -->|Approved| build[Build Package]
        build --> staging[Deploy to Staging]
        staging --> integration{Integration Tests}
        integration -->|Failed| code
        integration -->|Passed| prod[Deploy to Production]
        prod --> monitor[Monitor Performance]
        monitor -.-> rollback[Rollback if Issues]
        rollback --> code
    end
    
    classDef critical fill:#ff6b6b
    classDef process fill:#4ecdc4
    class rollback critical
    class start,prod process
```

Now, create a detailed flowchart for the following text while maintaining all relationships and processes:
""" + abstract

    messages = [
        {"role": "user", "content": prompt}
    ]

    client = InferenceClient(api_key="your api key")
    
    stream = client.chat.completions.create(
        model="mistralai/Mistral-7B-Instruct-v0.3",
        messages=messages,
        temperature=0.3,  # Lower temperature for more consistent output
        max_tokens=2048,
        top_p=0.9,
        stream=True
    )
    
    # Collect and process the response
    flowchart = ""
    for chunk in stream:
        if chunk.choices[0].delta.content:
            flowchart += chunk.choices[0].delta.content
            print(chunk.choices[0].delta.content, end="")
    
    return flowchart

# Function to validate mermaid syntax
def validate_mermaid_syntax(flowchart):
    # Basic validation checks
    required_elements = ['graph TD', '-->', '[', ']']
    for element in required_elements:
        if element not in flowchart:
            return False
    return True

In [11]:
flowchart_output = create_flowchart_prompt(abstract)

# Print the Mermaid.js flowchart syntax
print("\nGenerated Mermaid.js Flowchart:\n")
print(flowchart_output)

 Here's a detailed flowchart for the given text using Mermaid.js syntax:

```mermaid
graph TD
    subgraph Input: Initial Data Gathering
        start((Start)) --> data[Gather Data]
    end

    subgraph Generation: System Design
        data --> rag[Multimodal Retrieval Augmented Generation (RAG) Pipeline]
        rag --> gpt[GPT-4]
        gpt --> design[Generate System Design Documents]
    end

    subgraph Evaluation
        design --> evaluation{Evaluate Design Documents}
        evaluation -->|Passed| approve[Approve Design]
        evaluation -->|Failed| revise[Revise Design]
    end

    subgraph Output: Final Design Output
        revise --> rag
        rag --> gpt
        gpt --> design
        design --> approve
        approve --> output[Final Design Output]
    end

    subgraph SE Assistant
        data --> se_assistant[SE Assistant]
        se_assistant --> rag
        se_assistant --> evaluation
        se_assistant --> output
    end

    classDef process fill:#4ecdc4

In [16]:
import os

def create_mermaid_html_file(mermaid_code, filename="mermaid_flowchart.html"):
    """
    Create a static HTML file with Mermaid.js diagram
    """
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Mermaid Flowchart</title>
        <script src="https://cdn.jsdelivr.net/npm/mermaid/dist/mermaid.min.js"></script>
        <script>
            mermaid.initialize({{ 
                startOnLoad: true,
                theme: 'default'
            }});
        </script>
    </head>
    <body>
        <div class="mermaid">
            {mermaid_code}
        </div>
    </body>
    </html>
    """
    
    with open(filename, 'w') as f:
        f.write(html_content)
    
    print(f"HTML file created: {filename}")
    print("Please open this file in a web browser to view the flowchart.")

# Create the HTML file
create_mermaid_html_file(mermaid_code)

HTML file created: mermaid_flowchart.html
Please open this file in a web browser to view the flowchart.
